In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [ ]:
import xarray as xr
from xnoah.xarray import integrate
from xnoah.xcalc import centderiv

In [ ]:
iop = xr.open_dataset("../data/processed/iop.nc").squeeze().isel(lon=0,lat=8)

In [ ]:
moisture_budget = xr.Dataset(dict(
    qtend = integrate(iop.divq, axis='lev')/9.81 * 86400,
    prec = iop.PREC,
    LHF = iop.lhflx/2.51e6*86400
))

In [ ]:
imbalance = (moisture_budget.qtend-moisture_budget.prec + moisture_budget.LHF)

In [ ]:
plt.figure(dpi=100)

for key in moisture_budget.data_vars:
    moisture_budget[key].plot()
plt.legend()

In [ ]:
moisture_budget.mean()

Why is there an equilibrium imbalance of over 3 mm/day? Is it just that the vertical advection terms already implicitily include the evaporation? Do field campaigns do a similarly bad job? What does the temperature budget look like

In [ ]:
temperature_budget = xr.Dataset(dict(
    tend = integrate(iop.divT, axis='lev')/9.81 * 86400* 1004/2.51e6,
    prec = iop.PREC,
    SHF = iop.shflx/2.51e6*86400
))

In [ ]:
plt.figure(dpi=100)

for key in temperature_budget.data_vars:
    temperature_budget[key].plot()
    
plt.legend()

In [ ]:
temperature_budget.mean()

Note that I am not doing this for $s_L$ which is probably more approprite.

In [ ]:
import lib.thermo 

stat = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc")
force = xr.open_dataset("../data/processed/forcings.nc").isel(x=0,y=8)
rho = stat.RHO[-1]
dz = lib.thermo.get_dz(rho.z)
dm = rho * dz



In [ ]:
sl_budget = xr.Dataset(dict(
    tend = (dm*force.sl).sum('z') * 1004/2.51e6,
    prec = force.Prec,
    SHF = force.SHF/2.51e6*86400,
    QRAD= (dm*force.QRAD).sum('z')*1004/2.51e6
))

In [ ]:
d3 = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX_1280/coarse/3d/all.nc")
d2 = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX_1280/coarse/2d/all.nc")

In [ ]:
divwater = d3.divQV + d3.divQN + d3.divQP

(-(divwater * dm).sum('z').mean(['x', 'time'])/1000*86400).plot()
(-d2.Prec.mean(['x', 'time']) +
(d2.LHF.mean(['x', 'time'])*86400/2.51e6)).plot()

These budgets pretty closely match

In [ ]:
div1 = centderiv(d3.TABS * d3.U, 'x') + centderiv(d3.TABS*d3.V, 'y')
div2 = -d3.divTABS
adv = centderiv(d3.TABS,'x')*d3.U + centderiv(d3.TABS, 'y')*d3.V +\
      (centderiv(d3.TABS, 'z')) * d3.W


def integrate(div):
    return (div*dz*rho).sum('z').mean(['x', 'time'])

In [ ]:
advection_terms = ((adv + 9.81/1004 * d3.W) * dm).sum('z').mean(['x', 'time'])

(advection_terms * 1004/2.51e6*86400).plot()

(d2.Prec.mean(['x', 'time']) +
(d2.SHF.mean(['x', 'time'])*86400/2.51e6)+
 integrate(d3.QRAD)*1004/2.51e6
).plot()

The temperature budget does not match at all. it looks like some kind of scaling error. Shouldn't this quantity be small because temperature gradients are small?

In [ ]:
integrate(div1).plot()
integrate(div2).plot()
integrate(adv).plot()